In [1]:
import pandas as pd
import MeCab
from collections import Counter, defaultdict

In [2]:
tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /Users/yuki/foo/bar/user_dic2.dic")
tagger.parse("")

'EOS\n'

In [3]:
stop_words = ["♦","▪","⚪","・","◾",]

In [6]:
store_file = ["../pretest_aozora_dinner.csv",
             "../pretest_mitani_lunch.csv",
             "../pretest_mitani_dinner.csv"]

In [7]:
lunch_file = ["../pretest_sushitsuu_lunch.csv",
              "../pretest_sushiichi_lunch.csv",
              "../pretest_sawada_lunch.csv",
              "../pretest_matsukan_lunch.csv",
              "../pretest_kiyoda_lunch.csv",
              "../pretest_imamura_lunch.csv",
              "../pretest_hatsune_lunch.csv"
              ]
dinner_file = ["../pretest_sushitsuu_dinner.csv",
              "../pretest_sushiichi_dinner.csv",
              "../pretest_sawada_dinner.csv",
              "../pretest_matsukan_dinner.csv",
              "../pretest_kiyoda_dinner.csv",
              "../pretest_imamura_dinner.csv",
              "../pretest_hatsune_dinner.csv",
              ]

In [31]:
akazu = ["赤酢"]
nigiri = ["握り","にぎり","ニギリ"]
shari = ["シャリ","コメ","米","ライス","こめ","しゃり"]
maguro = ["まぐろ","マグロ","鮪","赤身","トロ","とろ"]

In [47]:
features = [akazu, nigiri, shari, maguro]

In [4]:
def tokenize_ja(text):
    node = tagger.parseToNode(str(text)) #分かち書きしたのが入っている。イテレータ
    while node:
        surface = node.surface.lower() #surfaceは単語の見た目　例：トろ
        feature = node.feature.split(",")
        hinshi = feature[0]
        genkei = feature[6]
        if surface not in stop_words and surface != "" and hinshi in ["形容詞", "動詞", "副詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"]:
            yield genkei
        elif surface not in stop_words and surface != "" and hinshi == "名詞":
            yield surface
        node = node.next

In [5]:
def tokenize(content):
    return [token for token in tokenize_ja(content)]

In [8]:
all_review_word_lunch = defaultdict(int)
all_review_word_dinner  =defaultdict(int)
num_word_lunch = 0
num_word_dinner = 0
for file in lunch_file:
    df=pd.read_csv(file)
    for review in df["lunch_review"]:
        list=tokenize(review)
        for l in list:
            all_review_word_lunch[l] += 1 #その単語の登場回数を1加える
            num_word_lunch += 1
for file in dinner_file:
    df=pd.read_csv(file)
    for review in df["dinner_review"]:
        list=tokenize(review)
        for l in list:
            all_review_word_dinner[l] += 1 #その単語の登場回数を1加える
            num_word_dinner += 1

In [11]:
counter_lunch = Counter(all_review_word_lunch).most_common()

In [12]:
counter_lunch

[('た', 4189),
 ('だ', 2947),
 ('する', 2402),
 ('ます', 2268),
 ('です', 2129),
 ('ない', 1373),
 ('いる', 1287),
 ('お', 1152),
 ('ある', 1019),
 ('の', 911),
 ('れる', 778),
 ('店', 636),
 ('鮨', 629),
 ('握り', 592),
 ('なる', 519),
 ('こと', 485),
 ('美味しい', 477),
 ('食べる', 451),
 ('さん', 451),
 ('シャリ', 432),
 ('思う', 409),
 ('ん', 407),
 ('この', 396),
 ('さ', 396),
 ('ネタ', 391),
 ('よう', 371),
 ('もの', 343),
 ('良い', 330),
 ('たい', 320),
 ('これ', 317),
 ('う', 303),
 ('くる', 269),
 ('０', 264),
 ('寿司', 260),
 ('熟成', 258),
 ('方', 257),
 ('ぬ', 256),
 ('その', 254),
 ('大将', 251),
 ('味', 245),
 ('中', 243),
 ('せる', 237),
 ('ランチ', 234),
 ('そう', 231),
 ('１', 231),
 ('られる', 226),
 ('鮪', 226),
 ('予約', 223),
 ('握る', 222),
 ('感じる', 213),
 ('炙る', 204),
 ('酢', 204),
 ('しまう', 201),
 ('行く', 199),
 ('また', 196),
 ('カウンター', 194),
 ('赤身', 191),
 ('くれる', 189),
 ('穴子', 188),
 ('中トロ', 185),
 ('貫', 184),
 ('私', 183),
 ('２', 181),
 ('前', 178),
 ('言う', 178),
 ('塩', 176),
 ('口', 175),
 ('こちら', 174),
 ('日', 173),
 ('ご', 172),
 ('入る', 164),
 ('香り', 

In [13]:
df = pd.read_csv(store_file[0])

In [ ]:
#以下黒越さんの要望資料作成用

In [45]:
dic = defaultdict(int)
word_times = 0 #全単語数
for review in df["dinner_review"]:
    token_list = tokenize(review) #各レビューを形態素解析
    for token in token_list:
        word_times += 1
        if (token in akazu) or (token in nigiri) or \
        (token in shari) or (token in maguro):
            dic[token] += 1

In [46]:
word_times

16730

In [43]:
dic

defaultdict(int,
            {'赤身': 41,
             '握り': 135,
             'シャリ': 129,
             '鮪': 26,
             'トロ': 6,
             '米': 20,
             'マグロ': 31,
             'まぐろ': 4,
             '赤酢': 1,
             'コメ': 1})

In [60]:
"赤身" in dic

True

In [80]:
l = []
for feature in features:
    for f in feature:
        if f in dic:
            ll = [feature[0], f, dic[f], int(10000*dic[f]/word_times)/100, 
                  int(10000*all_review_word_dinner[f]/num_word_dinner)/100,
                  int(100*(dic[f]/word_times)/(all_review_word_dinner[f]/num_word_dinner))/100]
            l.append(ll)

In [81]:
l

[['赤酢', '赤酢', 1, 0.0, 0.01, 0.54],
 ['握り', '握り', 135, 0.8, 0.58, 1.38],
 ['シャリ', 'シャリ', 129, 0.77, 0.37, 2.04],
 ['シャリ', 'コメ', 1, 0.0, 0.0, 11.4],
 ['シャリ', '米', 20, 0.11, 0.02, 4.95],
 ['まぐろ', 'まぐろ', 4, 0.02, 0.07, 0.31],
 ['まぐろ', 'マグロ', 31, 0.18, 0.2, 0.9],
 ['まぐろ', '鮪', 26, 0.15, 0.29, 0.52],
 ['まぐろ', '赤身', 41, 0.24, 0.22, 1.07],
 ['まぐろ', 'トロ', 6, 0.03, 0.09, 0.36]]

In [75]:
columns = ["軸","グループ単語","登場回数","登場率","他の店の登場率","他の店に比べて何倍登場しやすいか"]

In [82]:
df = pd.DataFrame(l, columns=columns)

In [83]:
df

,軸,グループ単語,登場回数,登場率,他の店の登場率,他の店に比べて何倍登場しやすいか
0,赤酢,赤酢,1,0.00,0.01,0.54
1,握り,握り,135,0.80,0.58,1.38
2,シャリ,シャリ,129,0.77,0.37,2.04
3,シャリ,コメ,1,0.00,0.00,11.40
4,シャリ,米,20,0.11,0.02,4.95
5,まぐろ,まぐろ,4,0.02,0.07,0.31
6,まぐろ,マグロ,31,0.18,0.20,0.90
7,まぐろ,鮪,26,0.15,0.29,0.52
8,まぐろ,赤身,41,0.24,0.22,1.07
9,まぐろ,トロ,6,0.03,0.09,0.36


In [88]:
l = []
for feature in features:
    n = 0
    s = 0
    m = ""
    for f in feature:
        n += dic[f]
        s += all_review_word_dinner[f]
        m += " " + f + " " + str(dic[f])
    ll = [feature[0],n, int(10000*n/word_times)/100, int(10000*s/num_word_dinner)/100,
         int(100*(n/word_times)/(s/num_word_dinner))/100, m]
    l.append(ll)

[['赤酢', 1, 0.0, 0.01, 0.54, ' 赤酢 1'],
 ['握り', 135, 0.8, 0.58, 1.37, ' 握り 135 にぎり 0 ニギリ 0'],
 ['シャリ', 150, 0.89, 0.4, 2.23, ' シャリ 129 コメ 1 米 20 ライス 0 こめ 0 しゃり 0'],
 ['まぐろ', 108, 0.64, 0.9, 0.71, ' まぐろ 4 マグロ 31 鮪 26 赤身 41 トロ 6 とろ 0']]

In [90]:
df = pd.DataFrame(l)

In [91]:
df

,0,1,2,3,4,5
0,赤酢,1,0.00,0.01,0.54,赤酢 1
1,握り,135,0.80,0.58,1.37,握り 135 にぎり 0 ニギリ 0
2,シャリ,150,0.89,0.40,2.23,シャリ 129 コメ 1 米 20 ライス 0 こめ 0 しゃり 0
3,まぐろ,108,0.64,0.90,0.71,まぐろ 4 マグロ 31 鮪 26 赤身 41 トロ 6 とろ 0


In [85]:
m = ""
m += "赤身"
m

'赤身'

In [ ]:
#以下提案資料作成用 未完成

In [18]:
df["dinner_review"][0].split("。")

['本日は美しきグルメOLと、2年ぶりの青空さんへ',
 'お任せで下記が出て来ました～金時草毛蟹の土佐ポン酢ジュレかけ秋刀魚肝醤油お刺身（かます・松川鰈・ナガスクジラ尾の身）蒸し鮑鰹の藁焼き北海道ししゃも※以下握りすみいかかすご赤身中トロ大トロこはだ小柱車エビいくら鯖バフンうにブリ穴子玉子焼きどの握りも圧巻でしたが、ブリ、穴子はその中でも群を抜いて美味しくいただけました',
 '日本酒と合わせながらほろ酔いで帰路につきます～ごちそうさまでした',
 '']

In [44]:
for review in df["dinner_review"]:
    li = review.split("。")
    for i in li:
        l = tokenize(i)
        if "まぐろ" and "美味しい" in l:
            print("t")

t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t


In [21]:
l

['すきやばし次郎さんで修業された方のお店と友人に紹介されていきました',
 '外国人のお客さんが多かったです',
 'つまみと握りは、特筆したかんじではなかったです',
 '少しシャリの酢が僕には強かったです',
 'しょうがない事だとは思うんですが、外国人の方の香水が強くて、食欲がなくなりました',
 '元々、香水の匂いが苦手なので残念な気持ちになりました',
 '最後にでた、卵は焼き上がりがふっくらでとても美味しかったです',
 'また、違ったシチュエーションで行くと違うのかな？']